# Creating the corpus
* Extracting a text corpus from Wikipedia made of plain text sentences
* selected so as to have a roughly balanced corpus in terms of training data (each target category should be associated with the same number of sentences). 

* Including 6 categories:
architects, mathematicians, painters, politicians, singers and writers.

***

* SCRIPT INPUT:
    
    * a number k of persons per category

    * a number n of sentences per person
    (persons whose wikipedia description is too short to have n sentences, should be ignored). 

* SCRIPT OUTPUT:

    * the text of the corresponding Wikipedia page
    
    * the corresponding data type (A or Z) and category
    
    * WikiData description
    
            ▶ Store these into a csv or a json file and save it on your hard drive

## Part 1
Create a list of persons you want to work with. Those persons
should fall into two main types: artists (A) and non artists (Z).
Singers, writers and painters are of type A while architects, politicians and mathematicians of type Z. For each category, select 30
persons of that category. So in total you should have a list of 180
persons (half of them are artists and half of them are not).
You can use the wikidata warehouse to find persons of the expected categories. More precisely, the Wikidata collection can be
filtered out using the SPARQL language and the following endpoint: https://query.wikidata.org/.
You can thus use the SPARQLwrapper python library to apply a
SPARQL query to the Wikidata warehouse and retrieve the required item identifiers.

In [1]:
# import modules
import wikipedia
from random import Random
from spacy.lang.en import English
from itertools import islice
import pandas as pd
import json
import wptools
import requests
import nltk
import random
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
keywords = ['architect', 'mathematician', 'painter', 'politician', 'singer', 'writer']

In [3]:
endpoint = SPARQLWrapper('https://query.wikidata.org/sparql')
endpoint.setReturnFormat(JSON)

In [4]:

def get_ids(keyword):

    result = requests.get('https://www.wikidata.org/w/api.php',
                        params={'format':'json',
                                'action':'wbsearchentities',
                                'search':keyword,
                                'language':'en'})
    result = result.json()

    key_id = result['search'][0]['id']

    query = '''SELECT ?personLabel
    WHERE
    {
    ?person wdt:P106 wd:'''+key_id+'''.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
    }
    '''

    endpoint.setQuery(query)
    results = endpoint.query()

    uri = json.loads(''.join(map(lambda x: x.decode(), results)))
    
    ids = []
    for person in uri['results']['bindings']:
        ids.append(person['personLabel']['value'])

    return ids 

In [5]:
ids = {}
for keyword in keywords:
    print('▶ keyword:', keyword)
    ids[keyword] = get_ids(keyword)

## Part 2
for each selected person, retrieve his.her Wikidata description and
Wikipedia page title. This can be done using the wikidata API
along with the wptools python library.

In [6]:
def get_title_and_description(id_):
    
    page = wptools.page(wikibase=id_)
    
    page.get_wikidata()
    title = page.data['title']
    description = page.data['description']
    
    return title, description

In [7]:
t,d = get_title_and_description('Q43799')
t,d

www.wikidata.org (wikidata) Q43799
www.wikidata.org (labels) Q36180|P7902|P1303|P8432|P214|Q14777025...
Johann Baptist Moser (en) data
{
  claims: <dict(26)> P19, P20, P27, P214, P227, P31, P569, P570, P...
  description: Austrian folk singer
  label: Johann Baptist Moser
  labels: <dict(37)> Q36180, P7902, P1303, P8432, P214, Q14777025,...
  modified: <dict(1)> wikidata
  requests: <list(2)> wikidata, labels
  title: Johann_Baptist_Moser
  what: human
  wikibase: Q43799
  wikidata: <dict(26)> place of birth (P19), place of death (P20),...
  wikidata_pageid: 45987
  wikidata_url: https://www.wikidata.org/wiki/Q43799
}


('Johann_Baptist_Moser', 'Austrian folk singer')

## Part 3
Once you have a list of wikipedia page titles, fetch (if it exists) the corresponding English wikipedia page, and extract the n first sentences of its content.

In [8]:
def get_content(title : str,
                n_sentences,
                sentence_tokenize=nltk.sent_tokenize):
    
    page = wikipedia.page(title)
    content = page.content
    sentences = [sent.strip() for sent in islice(sentence_tokenize(content), n_sentences)]
    
    return ' '.join(sentences)

In [9]:
c = get_content(t, 10)

In [13]:
def get_data(ids, n_people):
    data = {}
    for keyw,ppl_ids in ids.items():
        print('▶ keyw,ppl_ids[:n_people]', keyw,ppl_ids[:n_people])
        data[keyw] = []
        random.shuffle(ppl_ids)
        print('▶ len(ppl_ids)', len(ppl_ids))
        
        counter = 0
        for id_ in ppl_ids:
            if counter == n_people:
                break
            print('▶ counter', counter)
            print('▶ id_', id_)
            article = {}
            try:
                t, d = get_title_and_description(id_)
            except LookupError:
                print('▶ FAILED - LookupError')
                continue
            
            try:
                c = get_content(wikipedia.page(t), n_sentences=10)
            except (wikipedia.PageError, wikipedia.DisambiguationError):
                print('▶ FAILED - PageError / DisambiguationError')
                continue

            counter += 1

            article['title'] = t
            article['description'] = d
            article['content'] = c

            data[keyw].append(article)
    return data

In [22]:
%%time
data = get_data(ids=ids, n_people=30)

Laura Terracina (en) data
{
  claims: <dict(31)> P31, P21, P569, P735, P18, P214, P648, P1711,...
  description: Italian poet
  image: <list(1)> {'file': 'File:Laura terracina.PNG', 'kind': 'w...
  label: Laura Terracina
  labels: <dict(40)> Q49757, Q36180, Q37496426, P7902, Q652, P1986...
  modified: <dict(1)> wikidata
  requests: <list(3)> wikidata, labels, imageinfo
  title: Laura_Terracina
  what: human
  wikibase: Q6499496
  wikidata: <dict(31)> instance of (P31), sex or gender (P21), dat...
  wikidata_pageid: 6318554
  wikidata_url: https://www.wikidata.org/wiki/Q6499496
}
▶ counter 11
▶ id_ Q3017627
www.wikidata.org (wikidata) Q3017627
www.wikidata.org (labels) Q36180|P214|P3430|P2163|P646|P106|Q1860...
David Carpenter (writer) (en) data
{
  claims: <dict(22)> P214, P213, P31, P21, P19, P569, P735, P27, P...
  description: Canadian writer
  label: David Carpenter
  labels: <dict(31)> Q36180, P214, P3430, P2163, P646, P106, Q1860...
  modified: <dict(1)> wikidata
  requests: <lis

In [23]:
print(len(data['writer']),
        '\n',data.keys())

30 
 dict_keys(['architect', 'mathematician', 'painter', 'politician', 'singer', 'writer'])


In [24]:
# split categories
A = ['singer', 'writer', 'painter']
Z = ['architect', 'politician', 'mathematician']

A_cat = {a:data[a] for a in A}

Z_cat = {z:data[z] for z in Z}

data = {'A':A_cat, 'Z':Z_cat}

In [25]:
with open('data.json', 'w') as f:
    json.dump(data, f)